In [1]:
import plotly
import plotly.graph_objs as go
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()
from collections import Counter

from scipy.special import expit

from sklearn.model_selection import train_test_split,RepeatedStratifiedKFold,cross_val_score, GridSearchCV
from sklearn import tree

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix, roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
#from optbinning import BinningProcess, OptimalBinning # Para cáclulos WOE e IV
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from imblearn.over_sampling import RandomOverSampler

from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2, l1

from tabulate import tabulate

import statsmodels.api as sm
import statsmodels.formula.api as smf

2025-06-28 09:40:46.266322: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
data = pd.read_csv('vacuum_sensor_data.csv', sep = ';')
data

,momento,1-0005,5-0005,1-_006,5-_006,1-_007,5-_007,1-_008,5-_008,1-_010,...,1-_098,5-_098,1-_099,5-_099,1-0112,5-0112,1-0116,5-0116,1-0109,5-0109
0,1,-588.88,-580.21,-590.50,-582.25,-593.51,-585.46,-595.51,-587.47,-575.36,...,-588.17,-581.42,-594.26,-586.26,-589.46,-581.75,-592.92,-586.26,-593.93,-586.30
1,2,-588.38,-579.71,-590.80,-582.46,-593.80,-585.67,-595.55,-587.59,-574.11,...,-588.34,-581.63,-594.51,-586.26,-589.29,-581.58,-592.80,-586.26,-594.13,-586.51
2,3,-584.41,-575.74,-585.83,-577.66,-591.38,-583.21,-589.84,-578.04,-563.34,...,-588.17,-581.33,-592.88,-584.34,-588.50,-580.62,-590.21,-583.75,-593.68,-585.84
3,4,-573.73,-565.27,-570.31,-562.81,-578.91,-571.24,-578.61,-565.90,-555.00,...,-584.79,-576.87,-580.62,-568.15,-586.96,-578.75,-578.66,-571.19,-590.88,-582.63
4,5,-567.27,-558.55,-562.26,-554.55,-570.69,-562.93,-567.18,-557.51,-545.99,...,-579.11,-570.99,-572.11,-560.43,-583.45,-574.91,-571.06,-563.56,-587.38,-578.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,637,-237.18,-253.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
637,638,-237.10,-253.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638,639,-237.14,-253.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,640,-237.10,-253.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#rename column to have consistent naming 
import re

def clean_column(col):
    if col == 'momento':
        return col
    match = re.match(r"(\d)-_?(\d+)", col)
    if match:
        sensor, comp = match.groups()
        return f"{sensor}-{int(comp):04d}"
    return col  # fallback in case format is already correct

# Apply renaming
data.columns = [clean_column(col) for col in data.columns]

In [11]:
# drop rows(seconds) where data from any sensor is missing
data=data.dropna(axis=0)

In [12]:
data.shape

(533, 77)

In [13]:
"""
This script prepares sensor data for LSTM Autoencoder training and validation.
It extracts time series for each component (sensor 1 and sensor 5 readings),
splits them into training and validation sets, normalizes the data, and reshapes
it into sequences suitable for LSTM input.
"""

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import re



# Apply the time window filtering first
# The user specified 'momento 50-250', which corresponds to rows 49 to 249 (inclusive) in 0-indexed pandas.
# So, data.iloc[49:250] is correct for 201 timesteps.
data_full = data.iloc[49:250].copy() # Use .copy() to avoid SettingWithCopyWarning

# Define defective and normal control columns
defective_columns = ["1-0116", "5-0116", "1-0109", "5-0109"]
normal_control_columns = ["1-0008", "5-0008", "1-0064", "5-0064"]

# Separate momento column (if needed for later, but not for feature extraction)
momento_data = data["momento"]
sensor_data_raw = data.drop(columns=["momento"])

# Create a dictionary to store time series for each component
component_time_series = {}
# Extract unique component IDs from column names (e.g., '0005', '0006', etc.)
all_component_ids = sorted(list(set([col.split("-")[1] for col in sensor_data_raw.columns])))

# Iterate through each component ID and extract its sensor 1 and sensor 5 data
for comp_id in all_component_ids:
    s1_col = f"1-{comp_id}"
    s5_col = f"5-{comp_id}"
    
    # Check if both sensor columns exist for this component in the raw data
    if s1_col in sensor_data_raw.columns and s5_col in sensor_data_raw.columns:
        component_data = sensor_data_raw[[s1_col, s5_col]].values
        component_time_series[comp_id] = component_data
        print(f"Component {comp_id} data shape: {component_data.shape}") # Debugging line

# Split components into training and validation sets based on their IDs
defective_component_ids = sorted(list(set([col.split("-")[1] for col in defective_columns])))
normal_control_component_ids = sorted(list(set([col.split("-")[1] for col in normal_control_columns])))

# Training components are all components that are not in defective_component_ids or normal_control_component_ids
train_component_ids = [comp_id for comp_id in all_component_ids 
                       if comp_id not in defective_component_ids and comp_id not in normal_control_component_ids]

# Validation components are defective_component_ids + normal_control_component_ids
validation_component_ids = defective_component_ids + normal_control_component_ids

# Prepare training data
X_train_list = []
for comp_id in train_component_ids:
    if comp_id in component_time_series:
        X_train_list.append(component_time_series[comp_id])

# Concatenate all training time series into a single array for scaling
# This assumes all components have the same number of timesteps (rows).
# The `iloc[49:250]` slicing should ensure this, resulting in 201 timesteps.
X_train_flat = np.vstack(X_train_list)

# Normalize the training data (fit scaler on all normal component sensor readings)
scaler = MinMaxScaler()
X_train_scaled_flat = scaler.fit_transform(X_train_flat)

# Reshape back into sequences for each component
# The number of timesteps per component should be consistent (201).
TIMESTEPS_PER_COMPONENT = 201 

# Reconstruct X_train_sequences from the scaled flat array
X_train_sequences = []
current_idx = 0
for comp_id in train_component_ids:
    if comp_id in component_time_series:
        # Ensure the slice has the expected number of timesteps
        expected_end_idx = current_idx + TIMESTEPS_PER_COMPONENT
        if expected_end_idx <= len(X_train_scaled_flat):
            X_train_sequences.append(X_train_scaled_flat[current_idx : expected_end_idx])
            current_idx += TIMESTEPS_PER_COMPONENT
        else:
            print(f"Warning: Component {comp_id} has fewer than {TIMESTEPS_PER_COMPONENT} timesteps after slicing.")
            # Handle this case, e.g., by padding or skipping, or investigate why length is inconsistent.
            # For now, let's assume it's an error and will be caught if np.array fails.

X_train_sequences = np.array(X_train_sequences) # This is where the ValueError occurred previously

# Prepare validation data
X_validation_list = []
for comp_id in validation_component_ids:
    if comp_id in component_time_series:
        X_validation_list.append(component_time_series[comp_id])

# Scale validation data using the *same* scaler
X_validation_flat = np.vstack(X_validation_list)
X_validation_scaled_flat = scaler.transform(X_validation_flat)

# Reconstruct X_validation_sequences from the scaled flat array
X_validation_sequences = []
current_idx = 0
for comp_id in validation_component_ids:
    if comp_id in component_time_series:
        expected_end_idx = current_idx + TIMESTEPS_PER_COMPONENT
        if expected_end_idx <= len(X_validation_scaled_flat):
            X_validation_sequences.append(X_validation_scaled_flat[current_idx : expected_end_idx])
            current_idx += TIMESTEPS_PER_COMPONENT
        else:
            print(f"Warning: Validation component {comp_id} has fewer than {TIMESTEPS_PER_COMPONENT} timesteps after slicing.")

X_validation_sequences = np.array(X_validation_sequences)

print("Shape of X_train_sequences:", X_train_sequences.shape)
print("Shape of X_validation_sequences:", X_validation_sequences.shape)

# Save the processed data and component IDs
np.save("X_train_sequences.npy", X_train_sequences)
np.save("X_validation_sequences.npy", X_validation_sequences)
np.save("validation_component_ids.npy", np.array(validation_component_ids))




Component 0005 data shape: (533, 2)
Component 0006 data shape: (533, 2)
Component 0007 data shape: (533, 2)
Component 0008 data shape: (533, 2)
Component 0010 data shape: (533, 2)
Component 0011 data shape: (533, 2)
Component 0016 data shape: (533, 2)
Component 0018 data shape: (533, 2)
Component 0019 data shape: (533, 2)
Component 0021 data shape: (533, 2)
Component 0035 data shape: (533, 2)
Component 0038 data shape: (533, 2)
Component 0039 data shape: (533, 2)
Component 0041 data shape: (533, 2)
Component 0043 data shape: (533, 2)
Component 0045 data shape: (533, 2)
Component 0047 data shape: (533, 2)
Component 0050 data shape: (533, 2)
Component 0064 data shape: (533, 2)
Component 0065 data shape: (533, 2)
Component 0066 data shape: (533, 2)
Component 0067 data shape: (533, 2)
Component 0071 data shape: (533, 2)
Component 0076 data shape: (533, 2)
Component 0078 data shape: (533, 2)
Component 0079 data shape: (533, 2)
Component 0083 data shape: (533, 2)
Component 0084 data shape: (

In [16]:

import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping

# Load the prepared data
X_train_sequences = np.load("X_train_sequences.npy")

# Define the LSTM Autoencoder model
def create_lstm_autoencoder(timesteps, n_features, latent_dim=32):
    # Encoder
    inputs = Input(shape=(timesteps, n_features))
    encoded = LSTM(latent_dim, activation='relu')(inputs)
    
    # Decoder
    decoded = RepeatVector(timesteps)(encoded)
    decoded = LSTM(n_features, activation='relu', return_sequences=True)(decoded)
    
    autoencoder = Model(inputs, decoded)
    return autoencoder

timesteps = X_train_sequences.shape[1]
n_features = X_train_sequences.shape[2]

model = create_lstm_autoencoder(timesteps, n_features)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Define Early Stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
# For an autoencoder, the input and output are the same for training
history = model.fit(X_train_sequences, X_train_sequences, 
                    epochs=100, 
                    batch_size=32, 
                    validation_split=0.1, 
                    callbacks=[early_stopping],
                    verbose=1)

# Save the trained model in the native Keras format
model.save("lstm_autoencoder_model.keras")

print("Model training complete and saved as lstm_autoencoder_model.keras")




Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.2258 - val_loss: 0.1684
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step - loss: 0.2213 - val_loss: 0.1644
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - loss: 0.2167 - val_loss: 0.1604
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - loss: 0.2121 - val_loss: 0.1562
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - loss: 0.2074 - val_loss: 0.1521
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - loss: 0.2026 - val_loss: 0.1479
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - loss: 0.1977 - val_loss: 0.1437
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - loss: 0.1927 - val_loss: 0.1392
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - loss: 0.1876 - val_loss: 0.1347
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - loss: 0.1823 - val_loss: 0.1301
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step - loss: 0.1769 - val_loss: 0.1254
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step - loss: 0.1714

In [19]:

import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd

# Load the trained model
model = load_model("lstm_autoencoder_model.keras")

# Load the prepared data
X_train_sequences = np.load("X_train_sequences.npy")
X_validation_sequences = np.load("X_validation_sequences.npy")
validation_component_ids = np.load("validation_component_ids.npy")

# Predict reconstructions for training data
X_train_pred = model.predict(X_train_sequences)

# Calculate reconstruction errors for training data (per sequence)
train_reconstruction_errors = np.array([mean_squared_error(X_train_sequences[i].flatten(), X_train_pred[i].flatten())
                                        for i in range(len(X_train_sequences))])

# Predict reconstructions for validation data
X_validation_pred = model.predict(X_validation_sequences)

# Calculate reconstruction errors for validation data (per component)
# Each entry in X_validation_sequences corresponds to a component.
component_reconstruction_errors = []
for i in range(len(X_validation_sequences)):
    error = mean_squared_error(X_validation_sequences[i].flatten(), X_validation_pred[i].flatten())
    component_reconstruction_errors.append(error)

component_reconstruction_errors = np.array(component_reconstruction_errors)

# Determine an anomaly threshold based on training data reconstruction errors
anomaly_threshold = np.percentile(train_reconstruction_errors, 75)
print(f"Anomaly Threshold: {anomaly_threshold}")

# Identify anomalous components
anomalous_components_mask = component_reconstruction_errors > anomaly_threshold
anomalous_component_ids = validation_component_ids[anomalous_components_mask]

print(f"Number of anomalous components: {len(anomalous_component_ids)}")
print(f"Anomalous Component IDs: {anomalous_component_ids}")

# Visualize reconstruction errors for components
plt.figure(figsize=(10, 6))
plt.bar(validation_component_ids, component_reconstruction_errors, color='skyblue')
plt.axhline(anomaly_threshold, color='r', linestyle='--', label='Anomaly Threshold')
plt.xlabel('Component ID')
plt.ylabel('Reconstruction Error')
plt.title('Reconstruction Error per Validation Component')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.savefig('component_reconstruction_errors.png')
plt.close()

# Optional: Visualize distribution of training errors (same as before)
plt.figure(figsize=(12, 6))
plt.hist(train_reconstruction_errors, bins=50, density=True, alpha=0.6, label='Training Reconstruction Errors')
plt.axvline(anomaly_threshold, color='r', linestyle='--', label='Anomaly Threshold')
plt.title('Training Reconstruction Error Distribution')
plt.xlabel('Reconstruction Error')
plt.ylabel('Density')
plt.legend()
plt.savefig('training_reconstruction_error_distribution.png')
plt.close()




2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 630ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Anomaly Threshold: 0.07526985399337532
Number of anomalous components: 1
Anomalous Component IDs: ['0008']
